In [1]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[midje "1.9.1"])

{[com.rpl/specter "1.0.4" :exclusions [[org.clojure/clojure] [org.clojure/clojurescript]]] #{[riddley "0.1.12"]}, [mvxcvi/puget "1.0.2" :exclusions [[org.clojure/clojure]]] #{[mvxcvi/arrangement "1.1.1"] [fipp "0.6.10"]}, [mvxcvi/arrangement "1.1.1"] nil, [colorize "0.1.1" :exclusions [[org.clojure/clojure]]] nil, [org.clojure/tools.macro "0.1.5"] nil, [potemkin "0.4.3" :exclusions [[org.clojure/clojure]]] #{[clj-tuple "0.2.2"]}, [org.clojars.brenton/google-diff-match-patch "0.1"] nil, [swiss-arrows "1.0.0" :exclusions [[org.clojure/clojure]]] nil, [flare "0.2.9" :exclusions [[org.clojure/clojure]]] #{[org.clojars.brenton/google-diff-match-patch "0.1"]}, [org.tcrawley/dynapath "1.0.0"] nil, [marick/clojure-commons "3.0.0" :exclusions [[org.clojure/clojure]]] nil, [org.clojure/clojure "1.9.0"] #{[org.clojure/core.specs.alpha "0.1.24"] [org.clojure/spec.alpha "0.1.143"]}, [io.aviso/pretty "0.1.34"] nil, [org.clojure/math.combinatorics "0.1.4"] nil, [clj-time "0.14.2" :exclusions [[org.cl

In [2]:
(use 'clojure.pprint)
(use 'midje.repl)

Run `(doc midje)` for Midje usage.
Run `(doc midje-repl)` for descriptions of Midje repl functions.


## A Vampire Data Analysis Program for the FWPD

In [3]:
(def filename "resources/suspects.csv")

#'user/filename

In [4]:
(pprint (slurp filename))

"Edward Cullen,10\nBella Swan,0\nCharlie Swan,0\nJacob Black,3\nCarlisle Cullen,6\n"


In [5]:
; vector of keys used to create maps
(def vamp-keys [:name :glitter-index])

(defn str->int
    [str]
    (Integer/parseInt str))

; associate conversion function with each key
(def conversions {:name identity
                  :glitter-index str->int})
; takes a key and a value, and returns a converted value
(defn convert
    [vamp-key value]
    ((get conversions vamp-key) value))

#'user/convert

In [6]:
(convert :glitter-index "3")

3

In [7]:
(defn parse
    "Convert a CSV into rows of columns"
    [string]
    (map #(clojure.string/split % #",")
         (clojure.string/split string #"\n")))

#'user/parse

In [8]:
(pprint (parse (slurp filename)))

(["Edward Cullen" "10"]
 ["Bella Swan" "0"]
 ["Charlie Swan" "0"]
 ["Jacob Black" "3"]
 ["Carlisle Cullen" "6"])


In [9]:
(defn mapify
    "Return a seq of maps like {:name \"Edward Cullen\" :glitter-index 10}"
    [rows]
    (map (fn [unmapped-row]
             (reduce (fn [row-map [vamp-key value]]
                         (assoc row-map vamp-key (convert vamp-key value)))
                     {}
                     ; zip the keys with the cols in the row
                     (map vector vamp-keys unmapped-row)))
         rows))


#'user/mapify

In [10]:
(first (mapify (parse (slurp filename))))

{:name "Edward Cullen", :glitter-index 10}

In [11]:
(defn glitter-filter
    [minimum-glitter records]
    (filter #(>= (:glitter-index %) minimum-glitter) records))

#'user/glitter-filter

In [12]:
(def result (glitter-filter 3 (mapify (parse (slurp filename)))))
(pprint result)

({:name "Edward Cullen", :glitter-index 10}
 {:name "Jacob Black", :glitter-index 3}
 {:name "Carlisle Cullen", :glitter-index 6})


## Exercises

### Exercise 1
Turn the result of your glitter filter into a list of names.

In [13]:
(defn suspect-names [suspects]
    '())
(fact (suspect-names result) => '("Edward Cullen" "Jacob Black" "Carlisle Cullen"))


FAIL at (core.clj:3)
Expected:
("Edward Cullen" "Jacob Black" "Carlisle Cullen")
Actual:
()
Diffs: expected length of sequence is 3, actual length is 0.
                actual is missing 3 elements: ("Edward Cullen" "Jacob Black" "Carlisle Cullen")


false

### Exercise 2
Write a function, `append`, which will append a new suspect to your list of suspects.

In [14]:
(defn append [suspects suspect]
    '())

(let [suspects (parse (slurp filename))
      suspects-map (mapify suspects)
      new-suspect ["Alice Cullen" "7"]
      new-suspect-map {:name "Alice Cullen" :glitter-index 7}]
    (facts "about append"
           (fact "it can append to a list of vectors"
                 (append suspects new-suspect)
                 => '(
                         ["Edward Cullen"  "10"]
                         ["Bella Swan"      "0"]
                         ["Charlie Swan"    "0"]
                         ["Jacob Black"     "3"]
                         ["Carlisle Cullen" "6"]
                         ["Alice Cullen"    "7"]))
           (fact "it can append to a list of maps"
                 (append (mapify suspects) new-suspect-map)
                 => '(
                         {:name "Edward Cullen"   :glitter-index 10}
                         {:name "Bella Swan"      :glitter-index 0}
                         {:name "Charlie Swan"    :glitter-index 0}
                         {:name "Jacob Black"     :glitter-index 3}
                         {:name "Carlisle Cullen" :glitter-index 6}
                         {:name "Alice Cullen"    :glitter-index 7}))))


FAIL "about append - it can append to a list of vectors" at (core.clj:10)
Expected:
(["Edward Cullen" "10"]
 ["Bella Swan" "0"]
 ["Charlie Swan" "0"]
 ["Jacob Black" "3"]
 ["Carlisle Cullen" "6"]
 ["Alice Cullen" "7"])
Actual:
()
Diffs: expected length of sequence is 6, actual length is 0.
                actual is missing 6 elements: (["Edward Cullen" "10"] ["Bella Swan" "0"] ["Charlie Swan" "0"] ["Jacob Black" "3"] ["Carlisle Cullen" "6"] ["Alice Cullen" "7"])

FAIL "about append - it can append to a list of maps" at (core.clj:19)
Expected:
({:glitter-index 10 :name "Edward Cullen"}
 {:glitter-index 0 :name "Bella Swan"}
 {:glitter-index 0 :name "Charlie Swan"}
 {:glitter-index 3 :name "Jacob Black"}
 {:glitter-index 6 :name "Carlisle Cullen"}
 {:glitter-index 7 :name "Alice Cullen"})
Actual:
()
Diffs: expected length of sequence is 6, actual length is 0.
                actual is missing 6 elements: ({:name "Edward Cullen", :glitter-index 10} {:name "Bella Swan", :glitter-index 0} 

false

### Exercise 3

Write a function, `validate`, which will check that `:name` and `:glitter-index` are present when you append. The validate function should accept two arguments: a map of keywords to validating functions, similar to conversions, and the record to be validated.

In [15]:
(def validations {:name #(not (nil? %))
                  :glitter-index #(not (nil? %))})

(defn validate [validations record]
    false)

(let [suspects (parse (slurp filename))
      valid-record (first suspects)
      invalid-record ["Just a name"]]
    (facts "about validate"
           (fact "it returns true for valid records"
                 (validate validations valid-record) => true)
           (fact "it returns false for invalid records"
                 (validate validations invalid-record) => false)))


FAIL "about validate - it returns true for valid records" at (core.clj:12)
Expected:
true
Actual:
false


false

### Exercise 4

Write a function that will take your list of maps and convert it back to a CSV string. You’ll need to use the `clojure.string/join` function.

In [16]:
(defn to-csv [records]
    "")

(let [csv-string (slurp filename)
      records (parse csv-string)]
    (facts "about to-csv"
           (fact "it converts back to a csv string"
                 (to-csv records) => csv-string)))


FAIL "about to-csv - it converts back to a csv string" at (core.clj:8)
Expected:
"Edward Cullen,10\nBella Swan,0\nCharlie Swan,0\nJacob Black,3\nCarlisle Cullen,6\n"
Actual:
""
Diffs: strings have 1 difference (0% similarity)
                expected: "(Edward Cullen,10\nBella Swan,0\nCharlie Swan,0\nJacob Black,3\nCarlisle Cullen,6\n)"
                actual:   "(-----------------------------------------------------------------------------)"


false